In [1]:
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import datetime

Using TensorFlow backend.


In [2]:
import os

In [3]:
import plotly.graph_objects as go
import plotly.express as px

In [4]:
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout

In [5]:
def process(comp,n):
    company=comp
    s=datetime.date(2010,1,1)
    e=datetime.date.today() - datetime.timedelta(1)
    df = web.DataReader(company, data_source='yahoo', start=s, end=e)
    fig = go.Figure(data=[go.Candlestick(x=df.index,
                  open=df['Open'],
                  high=df['High'],
                  low=df['Low'],
                  close=df['Close'],
                  name='ohlc'                    )])
    fig.update_layout(xaxis_rangeslider_visible=False,
                      title=company,
                      xaxis_title="Date",
                      yaxis_title="Price",)
    print(fig.show)
    return fig,df

In [6]:
def closeT(df):
    #Create a new dataframe with only the 'Close column
    data = df.filter(['Close'])
    print(data)
    #Convert the dataframe to a numpy array
    dataset = data.values
    print(dataset)
    #Get the number of rows to train the model on
    training_data_len = math.ceil( len(dataset) * .8 )
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = scaler.fit_transform(dataset)
    #Create the training data set
    #Create the scaled training data set
    train_data = scaled_data[0:training_data_len , :]
    #Split the data into x_train and y_train data sets
    x_train = []
    y_train = []

    for i in range(60, len(train_data)):
      x_train.append(train_data[i-60:i, 0])
      y_train.append(train_data[i, 0])
      if i<= 61:
        print(x_train)
        print(y_train)
        print()
    #Convert the x_train and y_train to numpy arrays 
    x_train, y_train = np.array(x_train), np.array(y_train)
    
    #Reshape the data
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    
    #Build the LSTM model
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape= (x_train.shape[1], 1)))
    model.add(LSTM(50, return_sequences= False))
    model.add(Dense(25))
    model.add(Dense(1))
    
    #Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    if (not os.path.exists(f'stock_prediction-{ticker}.h5')):

        model.fit(x_train, y_train, batch_size=1, epochs=1)

        model.save(f'stock_prediction-{ticker}.h5')
        

In [7]:
def closeT(company,df):
    from keras.models import load_model
    from keras import backend as K
    #Create a new dataframe with only the 'Close column
    data = df.filter(['Close'])
    #print(data)
    
    #Convert the dataframe to a numpy array
    dataset = data.values
    #print(dataset)
    
    #Get the number of rows to train the model on
    training_data_len = math.ceil( len(dataset) * .8 )
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = scaler.fit_transform(dataset)
    
    #Create the training data set
    #Create the scaled training data set
    train_data = scaled_data[0:training_data_len , :]
    
    #Split the data into x_train and y_train data sets
    x_train = []
    y_train = []

    for i in range(60, len(train_data)):
      x_train.append(train_data[i-60:i, 0])
      y_train.append(train_data[i, 0])
      if i<= 61:
        print(x_train)
        print(y_train)
        print()
    #Convert the x_train and y_train to numpy arrays 
    x_train, y_train = np.array(x_train), np.array(y_train)
    #Reshape the data
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    
    #Build the LSTM model
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape= (x_train.shape[1], 1)))
    model.add(LSTM(50, return_sequences= False))
    model.add(Dense(25))
    model.add(Dense(1))
    
    #Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    if (not os.path.exists(f'stock_prediction-{company}.h5')):

        model.fit(x_train, y_train, batch_size=1, epochs=1)

        model.save(f'stock_prediction-{company}.h5')
        
    #Create the testing data set
    test_data = scaled_data[training_data_len - 60: , :]
    #Create the data sets x_test and y_test
    x_test = []
    y_test = dataset[training_data_len:, :]
    for i in range(60, len(test_data)):
        x_test.append(test_data[i-60:i, 0])
    
    #Convert the data to a numpy array
    x_test = np.array(x_test)
    
    #Reshape the data
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))
    #Get the models predicted price values 
    K.clear_session()
    model1 = load_model(f'stock_prediction-{company}.h5')
    predictions = model1.predict(x_test)
    predictions = scaler.inverse_transform(predictions)
    
    #Get the root mean squared error (RMSE)
    rmse=np.sqrt(np.mean(((predictions- y_test)**2)))
    print(rmse)#Plot the data
    train = data[:training_data_len]
    valid = data[training_data_len:]
    valid['Predictions'] = predictions
    
    #Visualize the data
    fig = go.Figure()
    
    # Create and style traces
    fig.add_trace(go.Scatter(x=df.index,y=df['Close'], name='close' ))
    fig.add_trace(go.Scatter(x=valid.index,y=valid['Predictions'],name='Close Predictions'))
    #print(fig.show)
    #print(valid)
    return valid,fig,scaler,model1

        

In [8]:
def predict(company,n,scaler,model):
            e=datetime.date.today()
            apple_quote = web.DataReader(company, data_source='yahoo', start='2015-01-01', end=e)

            #Create a new dataframe
            apple_quote.reset_index(inplace=True)
            
            #print(apple_quote)
            new_df  = pd.DataFrame(apple_quote[['Date','Close']])
            #print(new_df)
            #Get teh last 60 day closing price values and convert the dataframe to an array
            
            for z in range(0,n):
                new_df1=new_df.filter(['Close'])
                tomorrow=new_df['Date'].tail(1)
                #print(tomorrow)
                
                for date in tomorrow:
                    d1=date

                #print(tomorrow)
                #print(d1)
                if(d1.weekday()==4):
                    dd=3;
                else:
                    dd=1;
                last_60_days = new_df1[-60:].values
                #print("last_60_days",last_60_days)
            
                    #Scale the data to be values between 0 and 1
                last_60_days_scaled = scaler.transform(last_60_days)
                    #Create an empty list
                X_test = []
                    #Append teh past 60 days
                X_test.append(last_60_days_scaled)
                    #Convert the X_test data set to a numpy array
                X_test = np.array(X_test)
                    #Reshape the data
                X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
                    #Get the predicted scaled price
                pred_price = model.predict(X_test)
                    #undo the scaling 
                pred_price = scaler.inverse_transform(pred_price)
                #print(pred_price)
            
                for i in pred_price:
                    price=i
                    for j in i:
                        p1=j

                        new_row = {"Date":d1 + datetime.timedelta(days=dd),"Close":p1}
                        new_df=new_df.append(new_row,ignore_index=True)
                new_df2=new_df.tail(n)
                #print(new_df)
            
            fig = px.line(new_df, x='Date', y='Close')
            #print(fig.show)
            return new_df2,fig
            



In [9]:
from plotly.offline import plot
import json
import plotly
from flask import Flask, flash, redirect, render_template, request, url_for,session

app = Flask(__name__)

@app.route('/')
def forms():
    return render_template('/index.html')

@app.route('/index.html')
def index():
    return render_template('/index.html')

@app.route('/predict.html')
def cindex():
    return render_template('/predict.html')

@app.route('/proof.html')
def pindex():
    return render_template('proof.html')


@app.route('/result.html', methods = ['POST', 'GET'])
def result():
    
    #if request.method == 'POST':
            company=request.form["company"] 
            n1=request.form["period"] 
            n=int(n1)
            fig,df=process(company,n)
            df1=df.tail(92)
            fig_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
            valid,fig1,scaler,model=closeT(company,df)
            
            fig_json1 = json.dumps(fig1, cls=plotly.utils.PlotlyJSONEncoder)
            new_df2,fig2=predict(company,n,scaler,model)
            fig_json2 = json.dumps(fig2, cls=plotly.utils.PlotlyJSONEncoder)
            return render_template("/result.html",comp=company,period=n,table_df=[df1.to_html(table_id="myTable",classes=None,border=0,header="true")],tables=[new_df2.to_html(header="true",table_id="myTable1",border=0,index=False)],plot=fig_json,plot1=fig_json1,plot2=fig_json2)

if __name__ == "__main__":
    app.run(debug = True,use_reloader=False)



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Jun/2020 13:13:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2020 13:13:47] "GET /static/images/logo.png HTTP/1.1" 404 -
127.0.0.1 - - [23/Jun/2020 13:13:57] "GET /predict.html HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2020 13:13:57] "GET /static/images/logo.png HTTP/1.1" 404 -
127.0.0.1 - - [23/Jun/2020 13:13:57] "GET /static/images/Searchs_002.png HTTP/1.1" 404 -
127.0.0.1 - - [23/Jun/2020 13:13:59] "GET /proof.html HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2020 13:14:00] "GET /static/images/logo.png HTTP/1.1" 404 -
127.0.0.1 - - [23/Jun/2020 13:14:00] "GET /static/images/logo.png HTTP/1.1" 404 -
